In [177]:
import time
import itertools
import re
import random
import os
import pickle
import numpy as np

from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from lime.lime_text import LimeTextExplainer

import numpy as np
import pennylane as qml
from concurrent.futures import ThreadPoolExecutor


from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Nadam




from collections import defaultdict

import nltk  #This is to do lemmatization
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download("punkt_tab")
nltk.download("wordnet")
nltk.download("omw-1.4")


import tensorflow as tf

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\migue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\migue\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\migue\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# PART 0: DATA LOADING AND PREPROCESSING

In [178]:
X_text_all = None
y_all = None

def clean_text(text):
    cleaned = re.sub(r'<.*?>', '', text).lower()
        # Tokenize into words
    tokens = word_tokenize(cleaned)
    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Apply lemmatization to each token.
    # Here we use 'v' (verb) as the POS tag for simplicity.
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos='v') for token in tokens]
    return ' '.join(lemmatized_tokens)

def load_imdb_subset(
    num_samples=5000, 
    min_df=1, 
    max_features=15, 
    stopwords_option=True,
    stop_words = 'english'
):
    
    data = load_files(
        r"C:/Users/migue/Downloads/aclImdb_v1/aclImdb/train",
        categories=['pos','neg'], 
        encoding="utf-8", 
        decode_error="replace"                  
    )

    X_text_all, y_all = data.data, data.target


    X_text_all = [clean_text(txt) for txt in X_text_all]
    

    # Shuffle & truncate to num_samples
    full_idx = np.arange(len(X_text_all))
    #np.random.shuffle(full_idx)
    subset_idx = full_idx[:num_samples]
    
    X_text = [X_text_all[i] for i in subset_idx]
    
    y = y_all[subset_idx]

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_text, y, test_size=0.2, random_state=0
    )

    # Vectorizer: presence/absence
    if stopwords_option:
        vectorizer = CountVectorizer(
            binary=True, stop_words=stop_words, 
            min_df=min_df, max_features=max_features
        )
    else:
        vectorizer = CountVectorizer(
            binary=True, stop_words='english', 
            min_df=min_df, max_features=max_features
        )

    vectorizer.fit(X_train)
    return X_train, X_test, y_train, y_test, vectorizer



def train_NN_classifier(X_train, y_train, X_test, y_test, vectorizer):
    """
    Trains a neural network on the binary presence/absence of words.
    Returns the fitted model.
    """
    X_train_bow = vectorizer.transform(X_train)
    X_valid_bow = vectorizer.transform(X_test)
    input_dim = X_train_bow.shape[1]

    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),  # First hidden layer
        Dropout(0.3),  # Dropout with 30% probability
        Dense(64, activation='relu'),  # Second hidden layer
        Dropout(0.2),  # Dropout with 20% probability
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])
    
    # Compile the model
    model.compile(optimizer=Nadam(learning_rate = 0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(X_train_bow, y_train, epochs=30, batch_size=8, validation_data=(X_valid_bow, y_test), verbose=1, callbacks=[early_stopping])
    return model

def get_cached_NN(X_train, y_train, vectorizer, num_samples, max_features, stop_words, X_valid, y_valid):

    filename = f"cached_classifier_ns{num_samples}_mf{max_features}_sw{stop_words}_NN_classifier_seed0.pkl"
    if os.path.exists(filename):
        print("Loading cached logistic from", filename)
        with open(filename, 'rb') as f:
            clNN = pickle.load(f)
    else:
        print("No cached classifier found. Training a new one...")
        clNN = train_NN_classifier(X_train, y_train, X_valid, y_valid, vectorizer)
        with open(filename, 'wb') as f:
            pickle.dump(clNN, f)
        print("Cached classifier saved as", filename)
    return clNN

# I don't need to create train_NN_classifier_filt no? both _filt and not _filt are the same, so it should be fine withouth it

def get_cached_NN_filt(X_train_filt, y_train_filt, vectorizer_filt, num_samples, max_features, X_valid_filt, y_valid_filt):
    filename = f"cached_classifier_threshold_0.06_ns{num_samples}_mf{max_features}_sw_LIME_filtered_NN_classifier_seed0.pkl"
    if os.path.exists(filename):
        print("Loading cached logistic from", filename)
        with open(filename, 'rb') as f:
            clNN_filt = pickle.load(f)
    else:
        print("No cached classifier found. Training a new one...")
        clNN_filt = train_NN_classifier(X_train_filt, y_train_filt, X_valid_filt, y_valid_filt, vectorizer_filt)
        with open(filename, 'wb') as f:
            pickle.dump(clNN_filt, f)
        print("Cached classifier saved as", filename)
    return clNN_filt


# CLASSICAL LIME

In [179]:
def run_classical_lime(
    text_sample, clNN, vectorizer,  
    k_features=20, num_samples=50   
):
    
    """
    Runs classical LIME on a single text instance.
    Returns the top (word, weight) pairs.
    """
    class_names = ["negative", "positive"]
    explainer = LimeTextExplainer(class_names=class_names, feature_selection="auto")
    
    #@tf.function(reduce_retracing=True)
    def predict_proba(texts):
        bow = vectorizer.transform(texts) 
        #print('shaspe of box', bow.shape, ', text_sample:',text_sample, 'features: ', k_features, 'samples: ', num_samples)
        # print(bow)
        proba = clNN.predict(bow.toarray(), verbose=None)
        if proba.ndim == 1:  # If 1D, reshape to (num_samples, 1)
            proba = proba.reshape(-1, 1)
        #print('proba', proba, 'dimension', proba.shape, 'return', np.hstack((1 - proba, proba)))
        return np.column_stack((1 - proba, proba))  # Return probabilities for both classes
        
        

    explanation = explainer.explain_instance(
        text_sample,
        predict_proba,
        num_features=k_features,
        num_samples=num_samples 
    )
    return explanation.as_list() 

# EXPERIMENTAL ROUTINE

In [ ]:

def run_experiment(
    num_samples=500,
    min_df=1,
    max_features=15,

    stopwords_option=True,
    lime_num_samples=300,
    stop_words = 'english',
    max_features_filt = 20


):

    X_train, X_test, y_train, y_test, vectorizer = load_imdb_subset(
        num_samples=num_samples,
        min_df=min_df,
        max_features=max_features,
        stopwords_option=stopwords_option,
        stop_words = stop_words
        )

    clNN = get_cached_NN(X_train, y_train, vectorizer, num_samples, max_features, stop_words, X_test, y_test)

    X_test_bow = vectorizer.transform(X_test)
    y_test = y_test.reshape(-1, 1)
    test_acc = accuracy_score(y_test, clNN.predict(X_test_bow, verbose=None) > 0.5)

    instance_local_accuracies = []
    q_instance_local_accuracies = []

    print("Shape of y_train:", y_train.shape)
    print("Shape of y_test:", y_test.shape)
    X_all = X_train + X_test
    y_all = np.concatenate([y_train, y_test.ravel()])



    word_weights_unf = defaultdict(list)

    # lime_times = []
    
    for idx in range(len(X_all)):
        text_sample = X_all[idx]
        y_true = y_all[idx]

        #start_lime = time.time()

        explanation_lime_unfiltered = run_classical_lime(
            text_sample, clNN, vectorizer,
            k_features=max_features, num_samples=lime_num_samples
        )

        #bow = vectorizer.transform([text_sample])
        ###bin_features = bow.toarray()[0] #This is used in qlime


        # lime_time = time.time() - start_lime
        # lime_times.append(lime_time)


        contributions_unfiltered_abs = [(word, abs(score)) for word, score in explanation_lime_unfiltered]

        for word, weight in contributions_unfiltered_abs:

            word_weights_unf[word].append(weight)

        global_avg_weights_unf = {word: sum(weights) / len(weights) for word, weights in word_weights_unf.items()}

        threshold = 0.005


        rubish_words = {word: avg for word, avg in global_avg_weights_unf.items() if avg <= threshold}
        rubish_words_cleaned = {str(word): round(avg, 4) for word, avg in rubish_words.items()}

        filtered_words = {word: avg for word, avg in global_avg_weights_unf.items() if avg >= threshold}
        filtered_words_cleaned = {str(word): round(avg, 4) for word, avg in filtered_words.items()}

    #print('filtered_words', filtered_words_cleaned, 'rubish_words:', rubish_words_cleaned)

    features_filt = len(global_avg_weights_unf) - len(rubish_words_cleaned)


    X_train_filt, X_test_filt, y_train_filt, y_test_filt, vectorizer_filt = load_imdb_subset(
        num_samples=num_samples,
        min_df=min_df,
        max_features= max_features,
        stopwords_option=stopwords_option,
        stop_words = stop_words + list(rubish_words.keys())
        )

    clNN_filtered = get_cached_NN_filt(X_train_filt, y_train_filt, vectorizer_filt, num_samples, features_filt, X_test_filt, y_test_filt)

    # X_train_bow_filt = vectorizer_filt.transform(X_train_filt)
    # input_dim = X_train_bow_filt.shape[1]
    # print("Shape of training data:", X_train_filt.shape)
    # print("Shape of X_test_bow_filt:", X_test_bow_filt.shape)
    # print("Shape expected by model:", clNN_filtered.input_shape)
    # print("Shape of y_test_filt:", y_test_filt.shape)
    # print("Shape of predictions:", clNN_filtered.predict(X_test_bow_filt.toarray()).shape)
    # print("Model summary:")
    # clNN_filtered.summary()

    X_all_filt = X_train_filt + X_test_filt

    word_weights = defaultdict(list)
    for idx in range(len(X_all_filt)):

        if idx < len(X_all_filt):
            text_sample_filt = X_all_filt[idx]

            explanation_lime_filtered = run_classical_lime(
                text_sample_filt, clNN_filtered, vectorizer_filt,
                k_features=features_filt, num_samples=lime_num_samples
            )

            bow_filt = vectorizer_filt.transform([text_sample_filt])

            y_pred = clNN_filtered.predict(bow_filt.toarray(), verbose=None)[0].item()
            y_pred_label = 1 if y_pred >= 0.5 else 0

            instance_accuracy = int(y_pred_label == y_true)

            instance_local_accuracies.append(instance_accuracy)

            contributions_filtered_abs = [(word, abs(score)) for word, score in explanation_lime_filtered]

            for word, weight in contributions_filtered_abs:
                word_weights[word].append(weight)



    X_test_bow_filt = vectorizer_filt.transform(X_test_filt)
    test_acc_filt = accuracy_score(y_test_filt, clNN_filtered.predict(X_test_bow_filt.toarray(), verbose=None) > 0.5)




    results = {
        "global_acc": np.mean(test_acc), #Global accuracy before filtering
        "num_unf_words": len(word_weights_unf), 

        "num_filt_words": len(filtered_words_cleaned),
        #"local_accuracy": np.mean(instance_local_accuracies), #Lime local accuracies after filtering
        "global_acc_filtered": np.mean(test_acc_filt), #Lime global accuracy after filtering
        "list_deleted_words": rubish_words_cleaned,  #Words to delete Lime
        "list_filt_words": filtered_words_cleaned,  #Words to keep Lime
                }
    return results

#print(features_filt)

In [181]:
import pandas as pd
import sys, os

sys.path.append(os.getcwd())
sys.path.append(os.path.dirname(os.getcwd()))

if __name__ == "__main__":


    # Parameter grid to systematically vary certain settings # I still have to run ,2000, 3000 5000, 6000,7000,8000,9000,10000
    param_grid = {
        "num_samples": [5000],
        "max_features": [100],
        "stopwords_option": [True],
        "lime_num_samples": [1000],
        "stop_words": [['english']],
           
    }

    combos = list(itertools.product(*param_grid.values()))
    all_results = []

    for combo in combos:
        (num_samples_, max_features_, stopwords_, lime_samps_, stop_words_) = combo
        
        print("\n==================================")
        print(f"Running experiment with: "
              f"num_samples={num_samples_}, "
              f"max_features={max_features_}, "
              f"stopwords={stopwords_}, "
              f"lime_num_samples={lime_samps_}, "
              f"stop_words={stop_words_},")
        
        res = run_experiment(
            num_samples=num_samples_,
            max_features=max_features_,
            stopwords_option=stopwords_,
            lime_num_samples=lime_samps_,
            stop_words=stop_words_,
        )
        res_row = {
            "num_samples": num_samples_,
            "max_features": max_features_,
            "stopwords": stopwords_,
            "lime_num_samples": lime_samps_,
            #"local_accuracy": res["local_accuracy"],
            "global_acc": res["global_acc"],
            "global_acc_filtered": res["global_acc_filtered"],
            "num_unf_words": res["num_unf_words"], #Lime words
            "num_filt_words": res["num_filt_words"], #Lime words

            "list_deleted_words": res["list_deleted_words"],  #Lime words
            "list_filt_words": res["list_filt_words"],  #Lime words
        }

        #print("Results =>", res_row)
        all_results.append(res_row)

    # Save results to CSV
    df = pd.DataFrame(all_results)

    df.to_csv("results_expanded_flips.csv", index=False)

    print("\nAll done! Saved results to 'results_expanded_flips.csv'.")


Running experiment with: num_samples=5000, max_features=100, stopwords=True, lime_num_samples=1000, stop_words=['english'],
Loading cached logistic from cached_classifier_ns5000_mf100_sw['english']_NN_classifier_seed0.pkl
Shape of y_train: (4000,)
Shape of y_test: (1000, 1)
No cached classifier found. Training a new one...
Epoch 1/30


c:\Users\migue\miniconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5740 - loss: 0.6791 - val_accuracy: 0.6980 - val_loss: 0.5948
Epoch 2/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6794 - loss: 0.5919 - val_accuracy: 0.6990 - val_loss: 0.5791
Epoch 3/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7200 - loss: 0.5513 - val_accuracy: 0.6930 - val_loss: 0.5778
Epoch 4/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7310 - loss: 0.5295 - val_accuracy: 0.7110 - val_loss: 0.5697
Epoch 5/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7304 - loss: 0.5386 - val_accuracy: 0.7070 - val_loss: 0.5787
Epoch 6/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7446 - loss: 0.5104 - val_accuracy: 0.7160 - val_loss: 0.5649
Epoch 7/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7651 - loss: 0.4909 - val_accuracy: 0.7230 - val_loss: 0.5712
Epoch 8/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7786 - loss: 0.4806 - val_accuracy: 0.7190 - val_